# TP1 EDA UCDP GEDEvent 25.1


En este TP se muestra una visión general del dataset UCDP GEDEvent 25.1, de eventos de violencia organizada por actores armados, con resultado letal. Se pretende preparar estos datos para su uso en el entrenamiento de un modelo clasificador que pueda predecir el nivel de muertes estimado (best) agregado sobre una celda de 1,8° de latitud por 1,8° de longitud.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
from scipy import stats as st
from scipy.stats import describe
from scipy.stats import entropy

In [31]:
import pandas as pd
raw_GEDEvent = pd.read_csv('../datasets/GEDEvent_v25_1.csv')

/tmp/ipykernel_656842/1376209687.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_GEDEvent = pd.read_csv('../datasets/GEDEvent_v25_1.csv')



## EDA (Análisis Exploratorio de Datos)

### Columnas y tipos de datos

In [ ]:
raw_GEDEvent.columns

In [ ]:
# Armamos un nuevo dataset con menos columnas para el análisis
GEDEvent = raw_GEDEvent.drop(columns=['id', 'relid', 'year', 'active_year', 'code_status',
       'conflict_dset_id', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'number_of_sources',
       'source_article', 'source_office', 'source_date', 'source_headline',
       'source_original', 'where_prec', 'where_coordinates',
       'where_description', 'adm_2',
       'side_b_dset_id', 'side_b_new_id','geom_wkt', 'priogrid_gid', 'country_id',
       'event_clarity', 'date_prec', 'deaths_a',
       'deaths_b', 'deaths_unknown', 'high', 'low',
       'gwnoa', 'gwnob'], axis=1)

In [ ]:
GEDEvent.shape

In [ ]:
GEDEvent.columns


### Detalles de las columnas del dataset

#### Variables numéricas:

### Fecha de inicio y fin
* date_start
* date_end                       

### Coordenadas geográficas    
* latitude
* longitude

### Muertes del evento
* best:                                     La mejor estimación (más probable) del total 
                                            de muertes resultantes de un evento.
* deaths_civilians



#### Variables Categóricas:

### Continente, país y subdivisión     
* region:                                   Africa, Americas, Asia, Europe, Middle East
* country
* adm_1:                                    "Provincia" donde ocurrió el evento

### Tipo de violencia y actores
* type_of_violence:                         1 - conflicto estatal
                                            2 - conflicto no estatal
                                            3 - violencia unilateral
* side_a
* side_b


In [ ]:
# Reordeno las columnas para poder identificarlas y analizarlas más cómodamente 
nuevo_orden = ['date_start', 'date_end',                           # Fecha de inicio y fin
               'latitude', 'longitude',                            # Coordenadas geográficas    
               'best', 'deaths_civilians',                         # Muertes del evento
               'region', 'country', 'adm_1',                       # Continente, país y subdivisión
               'type_of_violence', 'side_a', 'side_b',             # Tipo de violencia y actores
]
GEDEvent = GEDEvent[nuevo_orden]

### Vista general del dataset

In [ ]:
GEDEvent.head(10)

In [ ]:
GEDEvent.iloc[0]

In [ ]:
GEDEvent.info()

In [ ]:
# Me fijo si hay algún valor raro en posición
GEDEvent[(GEDEvent['latitude'] < -90) | (GEDEvent['latitude'] > 90)]

### Ajuste de los tipos de datos

In [ ]:

         
# Numéricas temporales         
GEDEvent['date_start'] = pd.to_datetime(GEDEvent['date_start'])
GEDEvent['date_end'] = pd.to_datetime(GEDEvent['date_end'])

# Categóricas
GEDEvent['region'] = GEDEvent['region'].astype('category')
GEDEvent['country'] = GEDEvent['country'].astype('category')
GEDEvent['adm_1'] = GEDEvent['adm_1'].astype('category')
GEDEvent['type_of_violence'] = GEDEvent['type_of_violence'].astype('category')
GEDEvent['side_a'] = GEDEvent['side_a'].astype('category')
GEDEvent['side_b'] = GEDEvent['side_b'].astype('category')




In [ ]:
GEDEvent.info()

---

In [ ]:
GEDEvent[GEDEvent['best'] == '0']

## Cardinalidad: número de categorías únicas.

In [ ]:
GEDEvent.describe(include='category')

In [ ]:
# Top 10 Actor A
GEDEvent['side_a'].unique()

## Rango, frecuencia absoluta y frecuencia relativa: 
Número de veces que aparece cada categoría y su proporción con respecto al total.

In [ ]:

Ev= GEDEvent['side_a'].count()
print()
print("EDA variables categóricas.")
print()
print("side_a")
print()
v_counts = GEDEvent['side_a'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100

rango = v_counts.iloc[0] - GEDEvent['side_a'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print()

v_counts = GEDEvent['side_a'].value_counts().tail(1)
porcentajes = (v_counts / Ev) * 100

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print(f"Rango: {rango}")
print("-" * 60)
print()
print("side_b")
print()


v_counts = GEDEvent['side_b'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100
rango = v_counts.iloc[0] - GEDEvent['side_b'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print()

v_counts = GEDEvent['side_b'].value_counts().tail(1)
porcentajes = (v_counts / Ev) * 100

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print(f"Rango: {rango}")
print("-" * 60)
print()
print("region")
print()


v_counts = GEDEvent['region'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100
rango = v_counts.iloc[0] - GEDEvent['region'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print(f"Rango: {rango}")
print("-" * 60)
print()
print("country")
print()


v_counts = GEDEvent['country'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100
rango = v_counts.iloc[0] - GEDEvent['country'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print()

v_counts = GEDEvent['country'].value_counts().tail(1)
porcentajes = (v_counts / Ev) * 100

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print(f"Rango: {rango}")
print("-" * 60)
print()
print("adm_1")
print()


v_counts = GEDEvent['adm_1'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100
rango = v_counts.iloc[0] - GEDEvent['adm_1'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print()

v_counts = GEDEvent['adm_1'].value_counts().tail(1)
porcentajes = (v_counts / Ev) * 100

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print(f"Rango: {rango}")
print("-" * 60)
print()
print("type_of_violence")
print()


v_counts = GEDEvent['type_of_violence'].value_counts().head(10)
porcentajes = (v_counts / Ev) * 100
rango = v_counts.iloc[0] - GEDEvent['type_of_violence'].value_counts().tail(1).iloc[0]

for label, count, pct in zip(v_counts.index, v_counts.values, porcentajes):
    print(f"{label}: {count} ({pct:.2f}%)")
print()
print('1:conflicto estatal - 2:conflicto no estatal - 3:violencia unilateral')
print(f"Rango: {rango}")
print("-" * 60)


In [ ]:
top_10a = GEDEvent['side_a'].value_counts().head(10)
top_10a.plot(kind='bar', figsize=(10, 6), color='skyblue')

plt.show()

top_10b = GEDEvent['side_b'].value_counts().head(10)
top_10b.plot(kind='bar', figsize=(10, 6), color='salmon')

plt.show()

top_10a = GEDEvent['region'].value_counts().head(10)
top_10a.plot(kind='bar', figsize=(10, 6), color='lightblue')

plt.show()

top_10b = GEDEvent['country'].value_counts().head(10)
top_10b.plot(kind='bar', figsize=(10, 6), color='lightgreen')

plt.show()

top_10a = GEDEvent['adm_1'].value_counts().head(10)
top_10a.plot(kind='bar', figsize=(10, 6), color='orange')

plt.show()

top_10b = GEDEvent['type_of_violence'].value_counts().head(10)
top_10b.plot(kind='bar', figsize=(10, 6), color='purple')

plt.show()
print('1:conflicto estatal - 2:conflicto no estatal - 3:violencia unilateral')

## Entropía de Shannon

In [ ]:
for col in GEDEvent.select_dtypes(include=['object', 'category']):
    p = GEDEvent[col].value_counts(normalize=True)
    entropia = -np.sum(p * np.log2(p))
    print(f"Entropía de Shannon de '{col}': {entropia:.4f}")

In [ ]:

# Agrupar eventos por día y contar
eventos_por_dia = GEDEvent['date_start'].dt.date.value_counts().sort_index()

# Crear un DataFrame con las fechas y los conteos
df_eventos = pd.DataFrame({
    "fecha": eventos_por_dia.index,
    "cantidad_eventos": eventos_por_dia.values
})

# Graficar 
plt.figure(figsize=(14, 4))
sns.lineplot(data=df_eventos, x="fecha", y="cantidad_eventos", label="Eventos por día")

# Títulos y etiquetas
plt.title("Número de eventos por día")
plt.xlabel("Fecha")
#plt.yscale('log')
plt.ylabel("Cantidad de eventos")
plt.grid(True)
plt.show()



In [ ]:
# Análisis descriptivo de la variable 'best' por región
print("Análisis descriptivo de la variable 'best' por región")
regiones = GEDEvent['region'].unique()
print("-" * 60)

for region in regiones:
    print(f"\nRegión: {region}")
    print("-" * (len(region) + 8))


    region_best = GEDEvent[GEDEvent['region'] == region]['best']
    print(f"  Suma Total: {region_best.count()}")
    print(f"  Media: {region_best.mean():.2f}")
    print(f"  Mediana: {region_best.median():.2f}")
    print(f"  Moda: {region_best.mode().tolist()}")
    print(f"  Desviación Estándar: {region_best.std():.2f}")
    print(f"  Varianza: {region_best.var():.2f}")
    print(f"  Mínimo: {region_best.min()}")
    print(f"  Cuartil 1: {region_best.quantile(0.25):.2f}")
    print(f"  Cuartil 3: {region_best.quantile(0.75):.2f}")
    print(f"  Máximo: {region_best.max()}")
    print(f"  Rango Intercuartil: {(region_best.quantile(0.75)-region_best.quantile(0.25)):.2f}")
    print(f"  Asimetría: {region_best.skew():.2f}")
    print(f"  Curtosis: {region_best.kurtosis():.2f}")
    print("-" * (len(region) + 8))



In [ ]:
# Calcular muertes mensuales por región
# Agrupar por región y resamplear por mes, sumando las muertes
muertes_mensuales_por_region = GEDEvent.groupby('region').resample('M', on='date_start')['best'].sum().reset_index()
muertes_mensuales_por_region['date_start'] = muertes_mensuales_por_region['date_start'].dt.to_period('M').dt.to_timestamp()

# Promedio móvil 12 meses
muertes_mensuales_por_region['smoothed_best'] = muertes_mensuales_por_region.groupby('region')['best'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

plt.figure(figsize=(16, 9)) 
sns.lineplot(data=muertes_mensuales_por_region, x='date_start', y='smoothed_best', hue='region', linewidth=2)


plt.title('Mejor estimación muertes mensuales, promedio móvil 12 meses.')
plt.ylabel('best (log)')
plt.yscale('log')
plt.legend(title='Region')
plt.grid(True)
plt.show()

## Datos faltantes
Todos son adm_1 o "provincia" y la mayoría son de Europa. Todos tienen coordenadas geográficas, así que el dato, de existir, es recuperable.

In [ ]:

msno.matrix(GEDEvent)
plt.show()


tabla_cruzada = pd.crosstab(GEDEvent['region'], GEDEvent['adm_1'].isna(), normalize='index') * 100
print(tabla_cruzada.round(2))




In [ ]:
# Paso 1: Expandir eventos a muertes mensuales
print("Distribuyendo muertes mensualmente...")
filas = []
for _, evento in GEDEvent.iterrows():
    duracion = pd.date_range(evento['date_start'], evento['date_end'], freq='MS')
    if len(duracion) == 0:
        duracion = pd.date_range(evento['date_start'], periods=1, freq='MS')
    muertes_mensuales = evento['best'] / len(duracion)
    for d in duracion:
        filas.append({
            'año': d.year,
            'mes': d.month,
            'latitud': evento['latitude'],
            'longitud': evento['longitude'],
            'muertes': muertes_mensuales
        })

df = pd.DataFrame(filas)

In [ ]:
# Paso 2: Calcular índices de la cuadrícula 100x200
filas_cuadricula, columnas_cuadricula = 100, 200
df['fila'] = np.floor((-df['latitud'] + 90) / 1.8).astype(int)
df['columna'] = np.floor((df['longitud'] + 180) / 1.8).astype(int)
df['fila'] = df['fila'].clip(0, filas_cuadricula - 1)
df['columna'] = df['columna'].clip(0, columnas_cuadricula - 1)

In [ ]:
# Paso 3: Fechas únicas ordenadas
fechas = df[['año', 'mes']].drop_duplicates().sort_values(by=['año', 'mes']).to_records(index=False)

In [ ]:
# Paso 4: Crear grillas por mes (muertes acumuladas)
grillas_muertes = []
for año, mes in fechas:
    datos_mes = df[(df['año'] == año) & (df['mes'] == mes)]
    cuadricula = np.zeros((filas_cuadricula, columnas_cuadricula))
    grupo = datos_mes.groupby(['fila', 'columna'])['muertes'].sum()
    for (f, c), valor in grupo.items():
        cuadricula[int(f), int(c)] = valor
    grillas_muertes.append(cuadricula)



In [ ]:
# Paso 5: Clasificar las celdas en 4 niveles
def asignar_bin(cuadricula):
    niveles = np.full_like(cuadricula, 0)
    niveles[(cuadricula > 0) & (cuadricula <= 5)] = 1    # Bajo
    niveles[(cuadricula > 5) & (cuadricula <= 100)] = 2  # Medio
    niveles[cuadricula > 100] = 3                        # Alto
    return niveles

grillas_clasificadas = [asignar_bin(c) for c in grillas_muertes]

In [ ]:
# Paso 6: Configurar animación
etiquetas = ['Seguro', 'Bajo', 'Medio', 'Alto']
colores = ['white', 'yellow', 'orange', 'red']
cmapa = plt.matplotlib.colors.ListedColormap(colores)
imagen_mapa = mpimg.imread('img/World_location_map_(equirectangular_180)2.png')
fig, ax = plt.subplots(figsize=(12, 6))

def actualizar(frame):
    ax.clear()
    cuadricula = grillas_clasificadas[frame]
    año, mes = fechas[frame]
    ax.imshow(imagen_mapa, extent=[0, columnas_cuadricula, filas_cuadricula, 0])
    ax.imshow(cuadricula, cmap=cmapa, vmin=0, vmax=3, alpha=0.6)

    ax.set_title(f"Muertes por conflicto armado: {año}-{mes:02d}")
    ax.axis('off')



    
plt.close(fig)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

matplotlib.rcParams['animation.embed_limit'] = 200 * 1024 * 1024

# Paso 7: Animar
ani = FuncAnimation(fig, actualizar, frames=len(fechas), interval=150)
HTML(ani.to_jshtml())
